# 探索信号

In [3]:
from typing import List, Union

import numpy as np
import pandas as pd
import plotly.graph_objects as go

from signals.base import Metric, ChartConfig
from signals.indicators import lowpass_filter
from signals.utils import calculate_percentile_bands, add_percentile_bands

In [2]:
class RHODL(Metric):
    """Realized HODL Ratio"""

    @property
    def name(self) -> str:
        return "Realized HODL Ratio"

    @property
    def description(self) -> str:
        pass

    def __init__(
        self,
        data: pd.DataFrame,
        price_col: str = "btcusd",
        metric_cols: Union[str, List[str]] = "rhodl",
        signal_method: str = "percentile",  # percentile or zscore
        smooth_period: int = 30,
        percentile_period: int = 365 * 3,
        percentile_upper_band: float = 0.95,
        percentile_lower_band: float = 0.05,
        zscore_period: int = 365 * 2,
        zscore_upper_threshold: float = 2,
        zscore_lower_threshold: float = -2,
    ) -> None:
        """
        初始化指标类

        Args:
            data: 包含 NRPL 数据的 DataFrame
        """
        self.signal_method = signal_method
        self.smooth_period = smooth_period
        self.percentile_period = percentile_period
        self.percentile_upper_band = percentile_upper_band
        self.percentile_lower_band = percentile_lower_band
        self.zscore_period = zscore_period
        self.zscore_upper_threshold = zscore_upper_threshold
        self.zscore_lower_threshold = zscore_lower_threshold
        super().__init__(
            data,
            price_col,
            metric_cols,
            ChartConfig(rows=3, cols=1, height=800),
        )

    def generate_signals(self) -> None:

        metric_col = self.metric_cols[0]

        # 对数转换
        self.signals = self.data.copy()
        self.signals["log_rhodl"] = np.log(self.signals[metric_col])

        # 计算移动平滑
        self.signals["smooth_rhodl"] = lowpass_filter(
            self.signals["log_rhodl"], self.smooth_period
        )

        # 计算滚动百分位数
        self.signals["upper_band"] = (
            self.signals["smooth_rhodl"]
            .rolling(self.percentile_period)
            .quantile(self.percentile_upper_band)
        )
        self.signals["lower_band"] = (
            self.signals["smooth_rhodl"]
            .rolling(self.percentile_period)
            .quantile(self.percentile_lower_band)
        )

        # 计算滚动标准分数
        rolling_mean = self.signals["smooth_rhodl"].rolling(self.zscore_period).mean()
        rolling_std = self.signals["smooth_rhodl"].rolling(self.zscore_period).std()
        self.signals["rhodl_zscore"] = (
            self.signals["smooth_rhodl"] - rolling_mean
        ) / rolling_std

        # 生成信号
        if self.signal_method == "percentile":
            signal = np.where(
                self.signals["smooth_rhodl"] >= self.signals["upper_band"], 1, 0
            )
            signal = np.where(
                self.signals["smooth_rhodl"] <= self.signals["lower_band"], -1, signal
            )
            self.signals["signal"] = signal
        elif self.signal_method == "zscore":
            signal = np.where(
                self.signals["rhodl_zscore"] >= self.zscore_upper_threshold, 1, 0
            )
            signal = np.where(
                self.signals["rhodl_zscore"] <= self.zscore_lower_threshold, -1, signal
            )
            self.signals["signal"] = signal
        else:
            raise ValueError(f"Invalid signal method: {self.signal_method}")

    def _add_indicator_traces(self, fig: go.Figure) -> None:
        # 添加原始指标
        fig.add_trace(
            go.Scatter(
                x=self.signals.index,
                y=self.signals["log_rhodl"],
                name="RHODL(log)",
                line=dict(color="lightblue", width=1.5),
                opacity=0.5,
            ),
            row=2,
            col=1,
        )

        # 添加移动平滑指标
        fig.add_trace(
            go.Scatter(
                x=self.signals.index,
                y=self.signals["smooth_rhodl"],
                name="Smooth RHODL(log)",
                line=dict(color="royalblue", width=2),
            ),
            row=2,
            col=1,
        )

        # 添加百分位数通道
        fig.add_trace(
            go.Scatter(
                x=self.signals.index,
                y=self.signals["upper_band"],
                showlegend=False,
                line=dict(color="grey", width=1),
            ),
            row=2,
            col=1,
        )
        fig.add_trace(
            go.Scatter(
                x=self.signals.index,
                y=self.signals["lower_band"],
                showlegend=False,
                line=dict(color="grey", width=1),
                fill="tonexty",
                fillcolor="rgba(128, 128, 128, 0.1)",
            ),
            row=2,
            col=1,
        )

        fig.update_yaxes(row=2, col=1, title="RHODL", title_font=dict(size=14))

        # 添加滚动标准分数
        fig.add_trace(
            go.Scatter(
                x=self.signals.index,
                y=self.signals["rhodl_zscore"],
                name="Zscore",
                line=dict(color="red", width=2),
            ),
            row=3,
            col=1,
        )
        for level in [self.zscore_lower_threshold, self.zscore_upper_threshold]:
            fig.add_hline(
                level, line_dash="dot", line_color="grey", line_width=1, row=3, col=1
            )

        fig.update_yaxes(row=3, col=1, title="Zscore", title_font=dict(size=14))

In [25]:
class MVRVZscore(Metric):
    """MVRV Zscore"""

    @property
    def name(self) -> str:
        return "MVRV Zscore"

    @property
    def description(self) -> str:
        pass

    def __init__(
        self,
        data: pd.DataFrame,
        price_col: str = "btcusd",
        metric_cols: Union[str, List[str]] = "mvrv_zscore",
        smooth_period: int = 30,
        rolling_period: int = 365 * 3,
        upper_band_percentile: float = 0.99,
        lower_band_percentile: float = 0.01,
    ) -> None:
        self.smooth_period = smooth_period
        self.rolling_period = rolling_period
        self.upper_band_percentile = upper_band_percentile
        self.lower_band_percentile = lower_band_percentile
        super().__init__(data, price_col, metric_cols)

    def generate_signals(self) -> None:
        self.signals = calculate_percentile_bands(
            data=self.data.copy(),
            input_col=self.metric_cols[0],
            smooth_period=self.smooth_period,
            rolling_period=self.rolling_period,
            upper_band_percentile=self.upper_band_percentile,
            lower_band_percentile=self.lower_band_percentile,
        )

    def _add_indicator_traces(self, fig: go.Figure) -> None:
        metric_col = self.metric_cols[0]
        add_percentile_bands(
            fig=fig,
            data=self.signals,
            metric_col=metric_col,
            smooth_metric_col="smooth_" + metric_col,
            yaxis_title="MVRV Zscore",
            metric_name="MVRV Zscore",
            smooth_metric_name="Smooth MVRV Zscore",
        )

In [26]:
def get_all_data(filepath_ohlcv: str, filepath_metric: str) -> pd.DataFrame:
    ohlcv = pd.read_csv(filepath_ohlcv, index_col="datetime", parse_dates=True)
    metric = pd.read_csv(filepath_metric, index_col="datetime", parse_dates=True)

    return (
        pd.concat([ohlcv["close"], metric], axis=1, join="outer")
        .rename(columns={"close": "btcusd"})
        .ffill()
        .dropna()
    )


data = get_all_data(
    filepath_ohlcv="/users/scofield/quant-research/bitcoin_cycle/data/btcusd.csv",
    filepath_metric="/users/scofield/quant-research/bitcoin_cycle/data/mvrv_zscore.csv",
)

data

,btcusd,mvrv_zscore
datetime,,
2014-09-17,457.334015,0.428744
2014-09-18,424.440002,0.307226
2014-09-19,394.795990,0.172054
2014-09-20,408.903992,0.247914
2014-09-21,398.821014,0.203846
...,...,...
2025-05-25,109035.390625,2.624535
2025-05-26,109440.367188,2.638863
2025-05-27,108994.640625,2.620668


In [27]:
# metric = RHODL(
#     data,
#     signal_method="percentile",
#     percentile_period=365 * 3,
#     percentile_upper_band=0.95,
#     percentile_lower_band=0.05,
#     zscore_period=365 * 2,
#     zscore_upper_threshold=2.5,
#     zscore_lower_threshold=-2,
# )

metric = MVRVZscore(
    data,
    smooth_period=30,
    rolling_period=365 * 3,
    upper_band_percentile=0.99,
    lower_band_percentile=0.01,
)

metric.generate_signals()
fig = metric.generate_chart()
fig

In [28]:
# metric.signals